In [1]:
import numpy as np
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/open-problems-single-cell-perturbations/multiome_train.parquet
/kaggle/input/open-problems-single-cell-perturbations/multiome_obs_meta.csv
/kaggle/input/open-problems-single-cell-perturbations/sample_submission.csv
/kaggle/input/open-problems-single-cell-perturbations/adata_train.parquet
/kaggle/input/open-problems-single-cell-perturbations/multiome_var_meta.csv
/kaggle/input/open-problems-single-cell-perturbations/adata_obs_meta.csv
/kaggle/input/open-problems-single-cell-perturbations/id_map.csv
/kaggle/input/open-problems-single-cell-perturbations/de_train.parquet


# Configuration

In [3]:
import pandas as pd

In [16]:
dataset_path = "/kaggle/input/open-problems-single-cell-perturbations"

In [19]:
de_train = pd.read_parquet(f"{dataset_path}/de_train.parquet")
de_train.head(2)

,cell_type,sm_name,sm_lincs_id,SMILES,control,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,...,ZUP1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1
0,NK cells,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.104720,-0.077524,-1.625596,-0.144545,0.143555,...,-0.227781,-0.010752,-0.023881,0.674536,-0.453068,0.005164,-0.094959,0.034127,0.221377,0.368755
1,T cells CD4+,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.915953,-0.884380,0.371834,-0.081677,-0.498266,...,-0.494985,-0.303419,0.304955,-0.333905,-0.315516,-0.369626,-0.095079,0.704780,1.096702,-0.869887


In [20]:
id_map = pd.read_csv(f"{dataset_path}/id_map.csv")
id_map.head(2)

,id,cell_type,sm_name
0,0,B cells,5-(9-Isopropyl-8-methyl-2-morpholino-9H-purin-...
1,1,B cells,ABT-199 (GDC-0199)


In [21]:
submission = pd.read_csv(f"{dataset_path}/sample_submission.csv")z
submission.head(2)

,id,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,A4GALT,AAAS,AACS,AAGAB,...,ZUP1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Data preprocessing

In [22]:
de_train.isnull().sum()

cell_type      0
sm_name        0
sm_lincs_id    0
SMILES         0
control        0
              ..
ZXDB           0
ZXDC           0
ZYG11B         0
ZYX            0
ZZEF1          0
Length: 18216, dtype: int64

In [23]:
unique= de_train['sm_name'].unique()
unique, print(f"Count {len(unique)}")

Count 146


(array(['Clotrimazole', 'Mometasone Furoate', 'Idelalisib', 'Vandetanib',
        'Bosutinib', 'Ceritinib', 'Lamivudine', 'Crizotinib',
        'Cabozantinib', 'Flutamide', 'Dasatinib', 'Selumetinib',
        'Trametinib', 'ABT-199 (GDC-0199)', 'Oxybenzone', 'Vorinostat',
        'Raloxifene', 'Linagliptin', 'Lapatinib', 'Canertinib',
        'Disulfiram', 'Vardenafil', 'Palbociclib', 'Ricolinostat',
        'Dabrafenib', 'Proscillaridin A;Proscillaridin-A', 'IN1451',
        'Ixabepilone', 'CEP-18770 (Delanzomib)', 'RG7112', 'MK-5108',
        'Resminostat', 'IMD-0354', 'Alvocidib', 'LY2090314',
        'Methotrexate', 'LDN 193189', 'Tacalcitol', 'Colchicine', 'R428',
        'TL_HRAS26', 'BMS-387032', 'CGP 60474', 'TIE2 Kinase Inhibitor',
        'PD-0325901', 'Isoniazid', 'GSK-1070916', 'Masitinib',
        'Saracatinib', 'CC-401', 'Decitabine', 'Ketoconazole',
        'HYDROXYUREA', 'BAY 61-3606', 'Navitoclax', 'Porcn Inhibitor III',
        'GW843682X', 'Prednisolone', 'Tamatinib'

In [24]:
de_train['cell_type'].unique()

array(['NK cells', 'T cells CD4+', 'T cells CD8+', 'T regulatory cells',
       'B cells', 'Myeloid cells'], dtype=object)

In [25]:
de_train[de_train["control"] == True].count  

<bound method DataFrame.count of               cell_type     sm_name sm_lincs_id  \
104             B cells  Dabrafenib    LSM-6303   
105       Myeloid cells  Dabrafenib    LSM-6303   
106            NK cells  Dabrafenib    LSM-6303   
107        T cells CD4+  Dabrafenib    LSM-6303   
108        T cells CD8+  Dabrafenib    LSM-6303   
109  T regulatory cells  Dabrafenib    LSM-6303   
275             B cells  Belinostat   LSM-43181   
276       Myeloid cells  Belinostat   LSM-43181   
277            NK cells  Belinostat   LSM-43181   
278        T cells CD4+  Belinostat   LSM-43181   
279        T cells CD8+  Belinostat   LSM-43181   
280  T regulatory cells  Belinostat   LSM-43181   

                                                SMILES  control       A1BG  \
104  CC(C)(C)c1nc(-c2cccc(NS(=O)(=O)c3c(F)cccc3F)c2...     True   0.942197   
105  CC(C)(C)c1nc(-c2cccc(NS(=O)(=O)c3c(F)cccc3F)c2...     True   0.254567   
106  CC(C)(C)c1nc(-c2cccc(NS(=O)(=O)c3c(F)cccc3F)c2...     True  -0.1

In [26]:
# 设置随机种子
de_train = de_train.sample(frac=1.0, random_state=42)

In [27]:
de_train

,cell_type,sm_name,sm_lincs_id,SMILES,control,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,...,ZUP1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1
350,T regulatory cells,TPCA-1,LSM-1136,NC(=O)Nc1sc(-c2ccc(F)cc2)cc1C(N)=O,False,2.175274,1.219289,2.396227,2.885988,3.384956,...,-1.062199,1.178715,0.180416,0.755145,1.809688,1.644712,0.701406,0.137341,0.230949,0.179458
377,NK cells,Dovitinib,LSM-1127,CN1CCN(c2ccc3[nH]c(-c4c(N)c5c(F)cccc5[nH]c4=O)...,False,0.214531,-0.115171,0.326515,0.032289,0.220971,...,0.438939,0.216506,-0.108654,-0.063070,0.006721,-0.593161,-0.044022,0.475729,0.356555,-0.038020
163,T regulatory cells,Tacalcitol,LSM-45518,C=C1/C(=C\C=C2/CCC[C@]3(C)[C@@H]([C@H](C)CC[C@...,False,0.391257,-0.052922,0.540292,0.718991,1.083183,...,-1.755527,0.556352,0.089976,-0.000729,0.419802,1.020842,0.367313,0.486345,-0.856723,0.191009
609,T regulatory cells,Atorvastatin,LSM-5771,CC(C)c1c(C(=O)Nc2ccccc2)c(-c2ccccc2)c(-c2ccc(F...,False,-0.014372,-0.122464,-0.456366,-0.147894,-0.545382,...,-0.549987,-2.200925,0.359806,1.073983,0.356939,-0.029603,-0.528817,0.105138,0.491015,-0.979951
132,T cells CD8+,MK-5108,LSM-36812,O=C(O)[C@]1(Cc2cccc(Nc3nccs3)n2)CC[C@@H](Oc2cc...,False,0.094873,1.067610,-0.745980,1.083842,0.638504,...,0.279780,0.489431,0.147217,0.292313,0.029266,0.330628,1.038142,-0.194884,1.078132,0.057001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,T regulatory cells,Raloxifene,LSM-3425,O=C(c1ccc(OCCN2CCCCC2)cc1)c1c(-c2ccc(O)cc2)sc2...,False,1.080779,0.226991,2.440729,2.581894,1.773980,...,-0.276043,0.768273,1.351487,2.359651,1.470219,0.866639,0.467307,0.663700,-0.122574,-0.315800
106,NK cells,Dabrafenib,LSM-6303,CC(C)(C)c1nc(-c2cccc(NS(=O)(=O)c3c(F)cccc3F)c2...,True,-0.161665,0.269009,-1.770091,-0.198444,0.954753,...,0.131425,-1.112697,-0.589313,-0.660929,0.016530,0.319136,0.299521,-0.383452,0.267828,-1.100267
270,T regulatory cells,RN-486,LSM-47132,CN1CCN(c2ccc(Nc3cc(-c4cccc(-n5ccc6cc(C7CC7)cc(...,False,-0.304126,0.443651,0.688265,-0.101457,-0.005912,...,-0.855096,-0.499096,-0.211083,-0.495621,-0.233734,1.120530,-0.109096,0.128084,0.313468,-0.512444
435,NK cells,Doxorubicin,LSM-4062,COc1cccc2c1C(=O)c1c(O)c3c(c(O)c1C2=O)C[C@@](O)...,False,0.482294,-1.262924,-0.545405,-0.497739,-0.750201,...,0.444958,0.029279,0.337660,0.025025,0.035943,0.691171,0.352091,0.046012,-0.760812,-0.168719


# Feature Engineering

In [28]:
features_columns = ["cell_type", "sm_name"]
labels_columns=["cell_type","sm_name","sm_lincs_id","SMILES","control"]
labels = de_train.drop(columns=labels_columns)
features = pd.DataFrame(de_train, columns=features_columns)

In [29]:
features

,cell_type,sm_name
350,T regulatory cells,TPCA-1
377,NK cells,Dovitinib
163,T regulatory cells,Tacalcitol
609,T regulatory cells,Atorvastatin
132,T cells CD8+,MK-5108
...,...,...
71,T regulatory cells,Raloxifene
106,NK cells,Dabrafenib
270,T regulatory cells,RN-486
435,NK cells,Doxorubicin


In [30]:
labels

,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,A4GALT,AAAS,AACS,AAGAB,AAK1,...,ZUP1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1
350,2.175274,1.219289,2.396227,2.885988,3.384956,3.267348,0.134622,1.829941,-0.956191,-0.404039,...,-1.062199,1.178715,0.180416,0.755145,1.809688,1.644712,0.701406,0.137341,0.230949,0.179458
377,0.214531,-0.115171,0.326515,0.032289,0.220971,-0.463554,1.016849,0.153039,0.292684,0.038020,...,0.438939,0.216506,-0.108654,-0.063070,0.006721,-0.593161,-0.044022,0.475729,0.356555,-0.038020
163,0.391257,-0.052922,0.540292,0.718991,1.083183,1.021599,-0.097250,0.306833,-3.331719,0.235092,...,-1.755527,0.556352,0.089976,-0.000729,0.419802,1.020842,0.367313,0.486345,-0.856723,0.191009
609,-0.014372,-0.122464,-0.456366,-0.147894,-0.545382,-0.544709,0.282458,-0.431359,-0.364961,0.043123,...,-0.549987,-2.200925,0.359806,1.073983,0.356939,-0.029603,-0.528817,0.105138,0.491015,-0.979951
132,0.094873,1.067610,-0.745980,1.083842,0.638504,0.307649,-0.038108,0.772186,-0.736735,-0.462323,...,0.279780,0.489431,0.147217,0.292313,0.029266,0.330628,1.038142,-0.194884,1.078132,0.057001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,1.080779,0.226991,2.440729,2.581894,1.773980,1.989227,-0.074095,0.906555,2.214564,-0.182784,...,-0.276043,0.768273,1.351487,2.359651,1.470219,0.866639,0.467307,0.663700,-0.122574,-0.315800
106,-0.161665,0.269009,-1.770091,-0.198444,0.954753,1.155912,-0.312552,0.621897,-0.154055,0.331780,...,0.131425,-1.112697,-0.589313,-0.660929,0.016530,0.319136,0.299521,-0.383452,0.267828,-1.100267
270,-0.304126,0.443651,0.688265,-0.101457,-0.005912,-0.004914,-0.247434,0.281614,-0.915735,-0.032908,...,-0.855096,-0.499096,-0.211083,-0.495621,-0.233734,1.120530,-0.109096,0.128084,0.313468,-0.512444
435,0.482294,-1.262924,-0.545405,-0.497739,-0.750201,0.959704,0.527966,1.198200,-0.482740,0.635863,...,0.444958,0.029279,0.337660,0.025025,0.035943,0.691171,0.352091,0.046012,-0.760812,-0.168719
